In [1]:
import pandas as pd
import requests
from datetime import datetime
from sqlalchemy import create_engine, inspect
import io
import json
import os
import re

In [2]:
logs_vols = pd.read_csv(r'http://sc-e.fr/docs/logs_vols_2024-06-14.csv')
logs_vols.head(10)

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant
0,V05622564,A320_2562,2024-06-14,7.0,"{'temp': '15.0°C', 'pressure': '933.6 hPa', 'v...",0
1,V01097723,A380_2017,2024-06-14,9.7,"{'temp': '2.8°C', 'pressure': '975.9 hPa', 'vi...",0
2,V09506682,A340_2827,2024-06-14,7.5,"{'temp': '22.9°C', 'pressure': '881.6 hPa', 'v...",1
3,V06328529,A321_3033,2024-06-14,2.3,"{'temp': '-6.2°C', 'pressure': '988.7 hPa', 'v...",0
4,V01007218,CRJ900_3749,2024-06-14,8.4,"{'temp': '2.7°C', 'pressure': '936.7 hPa', 'vi...",0
5,V01180000,E170_6420,2024-06-14,8.1,"{'temp': '17.3°C', 'pressure': '877.3 hPa', 'v...",0
6,V06340711,A321_5105,2024-06-14,3.3,"{'temp': '-12.4°C', 'pressure': '925.8 hPa', '...",1
7,V06601310,B767_0906,2024-06-14,4.3,"{'temp': '22.5°C', 'pressure': '1007.8 hPa', '...",1
8,V04280543,B767_2803,2024-06-14,7.9,"{'temp': '-9.6°C', 'pressure': '961.8 hPa', 'v...",0
9,V08270143,E170_6913,2024-06-14,4.4,"{'temp': '-1.5°C', 'pressure': '952.5 hPa', 'v...",0


In [3]:
logs_vols.shape

(56, 6)

In [4]:
aero_e170_5551 = logs_vols[logs_vols['aero_linked'] == 'E170_5551']
aero_e170_5551

,ref_vol,aero_linked,jour_vol,time_en_air,sensor_data,etat_voyant


In [9]:
# date du jour
date_today = datetime.today().strftime('%Y-%m-%d')

In [46]:
# Configurer la connexion à la base de données
db_path = r'C:\Document\Python\Projet\Projet_3\Top_gun\bdd_sql\top_gun_data_1.db'
engine = create_engine(f'sqlite:///{db_path}')

date_defini = '2024-06-16'

In [47]:
# Fonction pour nettoyer les données
def clean_data(df):
    # Fonction pour corriger le format JSON
    def fix_json_format(x):
        return x.replace("'", '"')
    
    # Transformer la colonne 'sensor_data' de JSON en plusieurs colonnes
    df['sensor_data'] = df['sensor_data'].apply(lambda x: json.loads(fix_json_format(x)))
    sensor_data_df = pd.json_normalize(df['sensor_data'])
    df = pd.concat([df.drop(columns=['sensor_data']), sensor_data_df], axis=1)
    
    # Transformer la colonne 'jour_vol' au format date
    df['jour_vol'] = pd.to_datetime(df['jour_vol'], format='%Y-%m-%d', errors='coerce').dt.date
    
    # Nettoyer les colonnes 'temp', 'pressure', et 'vibrations'
    df['temp en °C'] = df['temp'].replace('°C', '', regex=True).astype(float)
    df['pressure en hPa'] = df['pressure'].replace('hPa', '', regex=True).astype(float)
    df['vibrations en m/s²'] = df['vibrations'].replace('m/s²', '', regex=True).astype(float).round(3)
    df.drop(columns=['temp', 'pressure', 'vibrations'], inplace=True)
    
    # Exclure les lignes où 'aero_linked' est 'E170_5551'
    df = df[df['aero_linked'] != 'E170_5551']

    return df

In [48]:
# Vérifier si une table existe déjà dans la base de données
def table_exists(engine, table_name):
    inspector = inspect(engine)
    return inspector.has_table(table_name)

# Fonction pour initialiser la base de données avec le fichier d'origine si la table n'existe pas
def initialize_database(engine):
    if not table_exists(engine, 'logs_vols'):
        logs_vols_data = pd.read_csv(r'C:\Document\Python\Projet\Projet_3\Top_gun\Datasets\df_logs_vols\logs_vols_2024-06-02.csv')
        
        # Nettoyer les données
        cleaned_data = clean_data(logs_vols_data)
        
        # Stocker les données dans la base de données SQL
        cleaned_data.to_sql(name='logs_vols', con=engine, if_exists='replace', index=False)
        print("Base de données initialisée avec le fichier d'origine.")
    else:
        print("La table 'logs_vols' existe déjà, aucune initialisation nécessaire.")

# Fonction pour récupérer les données logs_vols et les stocker dans la base de données SQL
def recup_log_vols(date, engine):
    url = f"http://sc-e.fr/docs/logs_vols_{date}.csv"

    response = requests.get(url)
    data = pd.read_csv(url)

    # Nettoyer les données
    data = clean_data(data)

    # Stocker les données dans la base de données SQL
    data.to_sql(name='logs_vols', con=engine, if_exists='append', index=False)
    print(f"Données de {date} ajoutées à la base de données.")

# Appeler la fonction pour initialiser la base de données avec le fichier d'origine si nécessaire
initialize_database(engine)

# Appeler la fonction pour récupérer et stocker les logs de vols
recup_log_vols(date_defini, engine)

La table 'logs_vols' existe déjà, aucune initialisation nécessaire.
Données de 2024-06-16 ajoutées à la base de données.
